In [ ]:
# 1
import numpy as np
import pandas as pd
import os

In [ ]:
# 2
import warnings
warnings.simplefilter("ignore", UserWarning)

import os
from tqdm import tqdm
import pickle
import scipy
import sys
import pandas as pd
import numpy as np
import IPython
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import librosa #Python package for music & audio files
import librosa.display
import librosa.display as lplt

from tensorflow import keras
from keras.models import Sequential
from IPython.display import Audio
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.feature_selection import RFECV,mutual_info_regression
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA

In [ ]:
final_data = pd.read_csv('/content/drive/MyDrive/features_3_sec.csv')

In [ ]:
final_data

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


In [ ]:
final_data = final_data.drop(labels='filename',axis=1)

In [ ]:
class_list = final_data.iloc[:, -1]
convertor = LabelEncoder()

y = convertor.fit_transform(class_list)

In [ ]:
# !!!
style = convertor.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
style

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [ ]:
# 3
scaler = StandardScaler()
X = scaler.fit_transform(np.array(final_data.iloc[:, :-1], dtype = float))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
def trainModel(model, epochs, optimizer):
    batch_size = 128
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                   metrics='accuracy'
    )
    return model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs,
                     batch_size=batch_size)

In [ ]:
X_test.shape

(3297, 58)

In [ ]:
# 4
model = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu", input_shape=(X_train.shape[1], )),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(256,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(10, activation="softmax"),

])
print(model.summary())
# model_history = trainModel(model=model, epochs=500, optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               30208     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=128)
print("The test loss is :",test_loss)
print("\nThe test Accuracy is :",test_accuracy*100)

26/26 [==============================] - 0s 2ms/step - loss: 0.4631 - accuracy: 0.9275
The test loss is : 0.46307528018951416

The test Accuracy is : 92.75098443031311


In [ ]:
# model.save('/content/drive/MyDrive/mgen')

In [ ]:
# 5
model = keras.models.load_model('/content/drive/MyDrive/mgen')

In [ ]:
# 6
def get_meta(p_t):
  y, sr = librosa.load(p_t)
  length = 66149
  chroma_stft_mean = np.mean(librosa.feature.chroma_stft(y = y, sr = sr))
  chroma_stft_var = np.var(librosa.feature.chroma_stft(y = y, sr = sr))

  rms_mean = np.mean(librosa.feature.rms(y = y))
  rms_var = np.var(librosa.feature.rms(y = y))

  spectral_centroid_mean = np.mean(librosa.feature.spectral_centroid(y = y, sr = sr))
  spectral_centroid_var = np.var(librosa.feature.spectral_centroid(y = y, sr = sr))

  spectral_bandwidth_mean = np.mean(librosa.feature.spectral_bandwidth(y = y, sr = sr))
  spectral_bandwidth_var = np.var(librosa.feature.spectral_bandwidth(y = y, sr = sr))

  rolloff_mean = np.mean(librosa.feature.spectral_rolloff(y = y, sr = sr))
  rolloff_var = np.var(librosa.feature.spectral_rolloff(y = y, sr = sr))

  zero_crossing_rate_mean = np.mean(librosa.feature.zero_crossing_rate(y))
  zero_crossing_rate_var = np.var(librosa.feature.zero_crossing_rate(y))

  harmony_mean = np.mean(librosa.effects.harmonic(y))
  harmony_var = np.var(librosa.effects.harmonic(y))

  perceptr_mean = np.mean(librosa.effects.percussive(y))
  perceptr_var = np.var(librosa.effects.percussive(y))

  tempo = librosa.feature.tempo(y = y, sr = sr)

  to_cl = {'length': length, 'chroma_stft_mean': chroma_stft_mean, 'chroma_stft_var': chroma_stft_var, 'rms_mean': rms_mean, 'rms_var': rms_var, 'spectral_centroid_mean': spectral_centroid_mean, 'spectral_centroid_var': spectral_centroid_var, 'spectral_bandwidth_mean': spectral_bandwidth_mean, 'spectral_bandwidth_var': spectral_bandwidth_var, 'rolloff_mean': rolloff_mean, 'rolloff_var': rolloff_var, 'zero_crossing_rate_mean': zero_crossing_rate_mean, 'zero_crossing_rate_var': zero_crossing_rate_var, 'harmony_mean': harmony_mean, 'harmony_var': harmony_var, 'perceptr_mean': perceptr_mean, 'perceptr_var': perceptr_var, 'tempo': tempo[0]}

  mfcc = librosa.feature.mfcc(y = y, sr = sr)

  i = 1
  for e in mfcc:
      s_m = 'mfcc' + str(i) + '_mean'
      s_v = 'mfcc' + str(i) + '_var'

      to_cl[s_m] = np.mean(e)
      to_cl[s_v] = np.var(e)
      i+=1
  to_mod = scaler.transform(np.array(list(to_cl.values())).reshape(1, 58))
  return to_mod

def get_style(p_t):
  return style[np.argmax(model(get_meta(p_t)))]

In [ ]:
p_t = '/content/drive/MyDrive/1.wav' #классика
# data, sr = librosa.load(p_t)

# IPython.display.Audio(data, rate=sr)

In [ ]:
get_style(p_t)

'classical'

In [ ]:
p_t = '/content/drive/MyDrive/2.wav' # поп
data, sr = librosa.load(p_t)

IPython.display.Audio(data, rate=sr)

In [ ]:
p_t = '/content/drive/MyDrive/2.wav'
get_style(p_t)

'pop'